In [1]:
!pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 24.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.0 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 

In [2]:
import torch
import os
import shutil
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset

2025-06-11 17:48:19.454761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749664099.656363      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749664099.720501      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Konfigurasi Model dan Path
model_name = "Qwen/Qwen2-1.5B-Instruct"
dataset_path = "/kaggle/input/sampah-dataset/combined_all_datasets.jsonl"
output_dir_training = "./qwen-1.5B-lora-p100"
output_dir_deployment = "./qwen_1.5B_chatbot_p100_final"

In [4]:
print("--- Langkah 1: Memuat Tokenizer dan Model ---")
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Tokenizer.pad_token diatur ke: {tokenizer.pad_token}")

# Pengaturan model untuk P100, memastikan semua menggunakan float16.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,             # Gunakan float16 untuk memuat bobot model
    bnb_4bit_compute_dtype=torch.float16,  # Tipe data komputasi untuk 4-bit, WAJIB float16 untuk P100
)

if model.config.pad_token_id is None or model.config.pad_token_id != tokenizer.pad_token_id:
    model.config.pad_token_id = tokenizer.pad_token_id
    print(f"Model.config.pad_token_id diatur ke: {model.config.pad_token_id}")

--- Langkah 1: Memuat Tokenizer dan Model ---


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Model.config.pad_token_id diatur ke: 151643


In [5]:
print("\n--- Langkah 2: Konfigurasi LoRA ---")
# Konfigurasi LoRA tidak perlu diubah
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(model, lora_config)
model.enable_input_require_grads()
model.print_trainable_parameters()


--- Langkah 2: Konfigurasi LoRA ---
trainable params: 18,464,768 || all params: 1,562,179,072 || trainable%: 1.1820


In [6]:
print("\n--- Langkah 3: Memuat dan Memproses Dataset ---")
dataset = load_dataset("json", data_files=dataset_path)
MAX_LENGTH = 512

def create_sft_datapoint(example):
    instruction = example['instruction']
    output_text = example['output']
    user_content = instruction
    messages = [{"role": "user", "content": user_content}, {"role": "assistant", "content": output_text}]
    full_text_str = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    model_input = tokenizer(full_text_str, max_length=MAX_LENGTH, truncation=True, padding="max_length", return_attention_mask=True)
    input_ids = model_input['input_ids']
    attention_mask = model_input['attention_mask']
    labels = list(input_ids)
    prompt_messages = [{"role": "user", "content": user_content}]
    prompt_only_str = tokenizer.apply_chat_template(prompt_messages, tokenize=False, add_generation_prompt=True)
    tokenized_prompt_ids = tokenizer(prompt_only_str, max_length=MAX_LENGTH, truncation=True, add_special_tokens=False)['input_ids']
    prompt_length = len(tokenized_prompt_ids)
    for i in range(prompt_length):
        if i < len(labels):
            labels[i] = -100
        else:
            break
    return {"input_ids": model_input['input_ids'], "attention_mask": model_input['attention_mask'], "labels": labels}

tokenized_dataset_train = dataset["train"].map(create_sft_datapoint, remove_columns=list(dataset["train"].features))
print("Dataset berhasil diproses.")


--- Langkah 3: Memuat dan Memproses Dataset ---


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2529 [00:00<?, ? examples/s]

Dataset berhasil diproses.


In [7]:
print("\n--- Langkah 4: Konfigurasi dan Menjalankan Training (Optimized for P100) ---")
training_args = TrainingArguments(
    output_dir=output_dir_training,
    # PENYESUAIAN UNTUK P100: Naikkan batch size untuk utilisasi VRAM 16GB yang lebih baik
    per_device_train_batch_size=2,
    # PENYESUAIAN: Turunkan accumulation steps untuk menjaga effective batch size (8*2 = 16)
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    num_train_epochs=5,
    learning_rate=4e-5,
    fp16=True,                      # Aktifkan mixed-precision training, P100 mendukung ini
    optim="paged_adamw_8bit",
    logging_steps=25,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    report_to="none",
    disable_tqdm=False,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset_train, data_collator=data_collator)

print("Memulai fine-tuning...")
trainer.train()
print("Fine-tuning selesai.")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



--- Langkah 4: Konfigurasi dan Menjalankan Training (Optimized for P100) ---
Memulai fine-tuning...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
25,2.214600
50,1.882700
75,1.655200
100,1.525600
125,1.482000
150,1.454300
175,1.368400
200,1.340200
225,1.333500
250,1.285500


Fine-tuning selesai.


In [8]:
print(f"Menyimpan adapter LoRA model ke: {output_dir_deployment}...")
# 'model' di sini adalah PeftModel Anda yang sudah dilatih
model.save_pretrained(output_dir_deployment)
print("Adapter LoRA berhasil disimpan.")

print(f"Menyimpan tokenizer ke: {output_dir_deployment}...")
tokenizer.save_pretrained(output_dir_deployment)
print("Tokenizer berhasil disimpan.")

Menyimpan adapter LoRA model ke: ./qwen_1.5B_chatbot_p100_final...
Adapter LoRA berhasil disimpan.
Menyimpan tokenizer ke: ./qwen_1.5B_chatbot_p100_final...
Tokenizer berhasil disimpan.


In [9]:
!zip -r /kaggle/working/hasil_training.zip /kaggle/working/qwen_1.5B_chatbot_p100_final

  adding: kaggle/working/qwen_1.5B_chatbot_p100_final/ (stored 0%)
  adding: kaggle/working/qwen_1.5B_chatbot_p100_final/vocab.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 61%)
  adding: kaggle/working/qwen_1.5B_chatbot_p100_final/tokenizer.json (deflated 81%)
  adding: kaggle/working/qwen_1.5B_chatbot_p100_final/merges.txt (deflated 57%)
  adding: kaggle/working/qwen_1.5B_chatbot_p100_final/tokenizer_config.json (deflated 65%)
  adding: kaggle/working/qwen_1.5B_chatbot_p100_final/README.md (deflated 66%)
  adding: kaggle/working/qwen_1.5B_chatbot_p100_final/adapter_model.safetensors (deflated 7%)
  adding: kaggle/working/qwen_1.5B_chatbot_p100_final/adapter_config.json (deflated 55%)
  adding: kaggle/working/qwen_1.5B_chatbot_p100_final/added_tokens.json (deflated 36%)
  adding: kaggle/working/qwen_1.5B_chatbot_p100_final/special_tokens_map.json (deflated 61%)
